# Spark 简单使用

In [1]:
from pyspark.sql import SparkSession

## 一、部署模式

### 1.1 Local 模式

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("jupyter-pyspark-demo") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/10/02 05:35:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1.2 Standalone 模式

In [3]:
spark = SparkSession.builder \
    .master("spark://master:7077") \
    .appName("jupyter-pyspark-standalone") \
    .getOrCreate()

25/10/02 05:36:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 1.3 YARN 模式

In [4]:
spark = SparkSession.builder \
    .master("yarn") \
    .appName("jupyter-pyspark-yarn") \
    .getOrCreate()

25/10/02 05:36:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## 二、文件系统

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("jupyter-pyspark-demo") \
    .getOrCreate()

25/10/02 05:36:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
# 打印统计信息
def show_stats(df):
    # 显示数据基本信息
    print("数据集行数:", df.count())
    print("数据集列数:", len(df.columns))
    # print("\n数据集结构:")
    # df.printSchema()
    # print("\n前5行数据:")
    # df.show(5)

### 2.1 本地文件系统

In [8]:
# 导入 CSV 文件 (使用本地文件系统路径)
df_local = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("file:///opt/share/spark_data/PRSA_data_2010.1.1-2014.12.31.csv")

In [9]:
show_stats(df_local)

数据集行数: 43824
数据集列数: 13


### 2.2 分布式文件系统

在使用 pyspark 读取 HDFS 文件之前，需要先把本地文件上传到 HDFS。

```bash
# 进入 bash
# docker compose exec -it spark bash
docker exec -it spark-hadoop-hive-master bash

# 激活 conda 环境
conda activate base

# 查看 hdfs 集群状态
hdfs dfsadmin -report

# 创建 hdfs 路径
hdfs dfs -mkdir -p /user/root

# 将本地数据上传到 hdfs
hdfs dfs -put /opt/share/spark_data/PRSA_data_2010.1.1-2014.12.31.csv /user/root/PRSA_data_2010.1.1-2014.12.31.csv

# 检查上传是否成功
hdfs dfs -ls /user/root/

# 删除 HDFS 文件
hdfs dfs -rm /user/root/PRSA_data_2010.1.1-2014.12.31.csv
```

In [11]:
df_hdfs = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("hdfs://master:9000/user/root/PRSA_data_2010.1.1-2014.12.31.csv")

In [12]:
show_stats(df_hdfs)

数据集行数: 43824
数据集列数: 13


## 三、分布式检查

检查文件是否分布式存储：

```bash
# 检查指定文件的健康状态
hdfs fsck /PRSA_data_2010.1.1-2014.12.31.csv -files -blocks -locations

# 获取 HDFS 集群配置参数
hdfs getconf -confKey dfs.blocksize

# 获取 HDFS 集群状态报告
hdfs dfsadmin -report

# 检查 yarn 服务的状态
yarn node -list 
```

## 四、使用 `spark-submit` 提交 Python 任务

脚本已在 `spark.py` 和 `spark-submit.sh` 中写好。

运行以下代码提交任务：

```bash
bash spark-submit.sh
```

## 五、使用 `hive -f` 提交 Hive 任务

两种用法：
- `hive -f [文件名]`
- `hive -e [SQL代码]`

脚本已在 `hive-submit.sh` 中写好。

运行以下代码提交任务：

```bash
bash hive-submit.sh
```